# Проверка различных значений

При создании выпукло-двойственной задачи мы дважды используем значения $x_0, z_0$. Один раз они являются ограничениями правой части, другой - коэффициентами.

При этом нам бы хотелось иметь возможность менять ограничеия правой части и при этом оставлять неизменными коэффициенты. Тогда получится встраивать такие ограничения в симплекс метод.

Попробуем заменить в ограничениях, которое отображает равенство целевых функций $x$ на $x_0 + \Delta x$ и $l$ на $l_0 + \Delta l$

In [1]:
from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray

import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
xa,xb = 0,1
za,zb = 0,1
grid_step = 0.1
xa_ext = xa - grid_step
xb_ext = xb + grid_step
za_ext = za - grid_step
zb_ext = zb + grid_step


x = np.arange(xa_ext, xb_ext, grid_step, dtype=np.double)
z = np.arange(za_ext, zb_ext, grid_step, dtype=np.double)
x,z = np.meshgrid(x, z)
y = x**2 + x * z + z**2

x = x.flatten()
z = z.flatten()
y = y.flatten()

mask = x+z <= 1+grid_step

xs = x[mask]
zs = z[mask]
ys = y[mask]

Для начала узнаем в как варьируется $l$

In [26]:
def f(xs, zs, ys, x, z, bx, bz):
    dim_p = len(xs)
    rhs_p = np.array([x,z, 1], dtype=np.double)
    dim_d = len(rhs_p) 

    s = CyClpSimplex()
    u = s.addVariable('u', dim_p)
    l = s.addVariable('l', dim_d)

    A_p = np.vstack([ys,
                         xs,
                         zs,
                         np.ones(dim_p)])

    A_p = np.matrix(A_p)

    b_p = CyLPArray(np.hstack([0,rhs_p]))
    b_d = CyLPArray(y)

    A_d = np.hstack([xs.reshape(-1, 1),
                     zs.reshape(-1, 1),
                     np.ones(len(xs)).reshape(-1, 1)])
    A_d = np.matrix(A_d)


    # A_d1 = np.matrix(np.vstack([-rhs_p,np.zeros((3,3))]))
    t = np.array([bx, bz, 1], dtype=np.double)
    A_d1 = np.matrix(np.vstack([t,np.zeros((3,3))]))

#     dx = bx - x_0
#     dz = bz - z_0
    t = np.array([x, z, 0], dtype=np.double)
    A_d2 = np.matrix(np.vstack([-t,np.zeros((3,3))]))

    #l_0 = np.array([1, 1, 1], dtype=np.double)
    
    #s += A_p*u - np.dot(A_d1,l_01) + A_d1*l + np.dot(A_d2,l_01) == b_p
    s += A_p*u + A_d1*l  == b_p
    for i in range(dim_p):
        s += u[i] >= 0

    s += A_d*l <= b_d

    s.objective = u[0]
    s.primal()

    cond = s.primalVariableSolution['u']
    yr = np.dot(cond, ys)
    
    xr = np.dot(cond, xs)
    zr = np.dot(cond, zs)
    return yr, x**2 + x * z + z**2, s.getStatusString(),xr,zr, s.primalVariableSolution['l']

In [32]:
r = f(xs,zs,ys,0.1,0.1,0.1,0.1)
r

(0.07333333333333336,
 0.030000000000000006,
 'optimal',
 0.1,
 0.1,
 array([ 0.        ,  0.        , -0.07333333]))

In [5]:
r = f(xs,zs,ys,0.3,0.4,np.array([-0.54338235,  0.  , -0.04433824]),np.array([0., 0., 0.31470588]),0.3,0.4)
r

(0.42470588235294127,
 0.37,
 'optimal',
 0.3,
 0.4,
 array([ 0.        ,  0.        , -0.42470588]))

In [6]:
r = f(xs,zs,ys_pos,ys_neg,0.5,0.5,np.array([ 1.5,  1.4, -0.7]),np.array([-0.9, -0.9,  0.4]),0.5,0.5)
r

NameError: name 'ys_pos' is not defined

In [ ]:
r = f(xs,zs,ys_pos,ys_neg,0.5,0.5,r[-2],r[-1],0.5,0.5)
r

---

In [ ]:
r = f(xs,zs,ys_pos,ys_neg,0.1,0.1,np.array([1,1,1]),np.array([1,1,1]),1,1)
r

In [ ]:
r = f(xs,zs,ys_pos,ys_neg,0.1,0.1,np.array([1,1,1]),np.array([1,1,1]),1,1)
r

---

In [34]:
for x in np.arange(0.1,1,0.1):
    for y in np.arange(0.1,1,0.1):
        if x+y <= 1:
            r = f(xs,zs,ys,x,y,1,1)
            arr = [*r[-2],*r[-1]]
            print("{:2.1f} {:2.1f} :fx= {:+.4f}, {:+.4f}, \u0394fx: {:.4f}; l \u2208 [{:.2f},{:.2f}]".
                  format(x,y,r[0],x*y,abs(r[0]-x*y),min(arr),max(arr)))

TypeError: 'numpy.float64' object is not iterable

In [ ]:
for x in np.arange(0.1,1,0.1):
    for y in np.arange(0.1,1,0.1):
        if x+y <= 1:
            r = f(xs,zs,ys_pos,ys_neg,x,y,np.array([0,0,0]),np.array([0,0,0]),x,y)
            arr = [*r[-2],*r[-1]]
            print("{:2.1f} {:2.1f} :fx= {:+.4f}, {:+.4f}, \u0394fx: {:.4f}; l \u2208 [{:.2f},{:.2f}]".
                  format(x,y,r[0],x*y,abs(r[0]-x*y),min(arr),max(arr)))